# Web Scraping - Lab

## Introduction

Now that you've seen a more extensive example of developing a web scraping script, it's time to further practice and formalize that knowledge by writing functions to parse specific pieces of information from the web page and then synthesizing these into a larger loop that will iterate over successive web pages in order to build a complete dataset.

## Objectives

You will be able to:

* Navigate HTML documents using Beautiful Soup's children and sibling relations
* Select specific elements from HTML using Beautiful Soup
* Use regular expressions to extract items with a certain pattern within Beautiful Soup
* Determine the pagination scheme of a website and scrape multiple pages

## Lab Overview

This lab will build upon the previous lesson. In the end, you'll look to write a script that will iterate over all of the pages for the demo site and extract the title, price, star rating and availability of each book listed. Building up to that, you'll formalize the concepts from the lesson by writing functions that will extract a list of each of these features for each web page. You'll then combine these functions into the full script which will look something like this:  

```python
df = pd.DataFrame()
for i in range(2,51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    new_titles = retrieve_titles(soup)
    new_star_ratings = retrieve_ratings(soup)
    new_prices = retrieve_prices(soup)
    new_avails = retrieve_avails(soup)
    ...
 ```

## Retrieving Titles

To start, write a function that extracts the titles of the books on a given page. The input for the function should be the `soup` for the HTML of the page.

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [17]:
def get_to_books_container(soup):
    # go to the alert div
    alert_div = soup.find('div', class_='alert alert-warning')
#     print(alert_div.name)
    alert_div_next = alert_div.nextSibling
    while alert_div_next.name != 'div':
#         print(alert_div_next.name)
        # go to the nextSibling to skip over white/text space
        alert_div_next = alert_div_next.nextSibling
    
    return alert_div_next
    

In [18]:
def retrieve_titles(soup):
    
    books_container = get_to_books_container(soup)
    
    # get the titles from the <a title='the title'> within the <h3>
    titles = [h3.find('a').attrs['title'] for h3 in books_container.findAll('h3')]
    
    return titles

## Retrieve Ratings

Next, write a similar function to retrieve the star ratings on a given page. Again, the function should take in the `soup` from the given HTML page and return a list of the star ratings for the books. These star ratings should be formatted as integers.

In [19]:
def retrieve_ratings(soup):
    
    books_container = get_to_books_container(soup)
    
    # use a regular expression to find all the <p> with a class including 'star-rating'
    regex = re.compile("star-rating (.*)")
    star_ratings = []
    for p in books_container.findAll('p', {"class" : regex}):
        star_ratings.append(p.attrs['class'][-1])

    # Convert the star ratings to Int
    star_dict = {'One': 1, 'Two': 2, 'Three':3, 'Four': 4, 'Five':5} 
    # Manually create a dictionary to translate to numeric
    star_ratings = [star_dict[s] for s in star_ratings]
    
    return star_ratings
    

## Retrieve Prices

Now write a function to retrieve the prices on a given page. The function should take in the `soup` from the given page and return a list of prices formatted as floats.

In [20]:
def retrieve_prices(soup):
    
    books_container = get_to_books_container(soup)
    
    prices = [p.text for p in books_container.findAll('p', class_="price_color")]
    
    #remove the pound sign and convert to a float
    prices = [float(p[1:]) for p in prices]
    
    return prices
    

## Retrieve Availability

Write a function to retrieve whether each book is available or not. The function should take in the `soup` from a given html page and return a list of the availability for each book.

In [21]:
def retrieve_availabilities(soup):
    
    books_container = get_to_books_container(soup)
    
    avail = [a.text.strip() for a in books_container.findAll('p', class_="instock availability")]
    
    return avail

## Create a Script to Retrieve All the Books From All 50 Pages

Finally, write a script to retrieve all of the information from all 50 pages of the books.toscrape.com website. 

In [29]:
def get_page_info(soup):
    
    titles = retrieve_titles(soup)
    ratings = retrieve_ratings(soup)
    prices = retrieve_prices(soup)
    avails = retrieve_availabilities(soup)
    
    df = pd.DataFrame([titles, ratings, prices, avails]).transpose()
    
    return df

In [32]:
# get the first page
html_page = requests.get('http://books.toscrape.com/') # Make a get request to retrieve the page
soup = BeautifulSoup(html_page.content, 'html.parser') # Pass the page contents to beautiful soup for parsing

df = get_page_info(soup)


for page in range(2,51):
    # get the next page
    html_page = requests.get(f'http://books.toscrape.com/catalogue/page-{page}.html') # Make a get request to retrieve the page
    soup = BeautifulSoup(html_page.content, 'html.parser') # Pass the page contents to beautiful soup for parsing
    
    df_new_page = get_page_info(soup)
    
    df = pd.concat([df, df_new_page])
    
# add the column names
df.columns = ['Title', 'Rating', 'Price', 'Avail']
    
df

,Title,Rating,Price,Avail
0,A Light in the Attic,3,51.77,In stock
1,Tipping the Velvet,1,53.74,In stock
2,Soumission,1,50.1,In stock
3,Sharp Objects,4,47.82,In stock
4,Sapiens: A Brief History of Humankind,5,54.23,In stock
...,...,...,...,...
15,Alice in Wonderland (Alice's Adventures in Won...,1,55.53,In stock
16,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",4,57.06,In stock
17,A Spy's Devotion (The Regency Spies of London #1),5,16.97,In stock
18,1st to Die (Women's Murder Club #1),1,53.98,In stock


## Level-Up: Write a new version of the script you just wrote. 

If you used URL hacking to generate each successive page URL, instead write a function that retrieves the link from the `"next"` button at the bottom of the page. Conversely, if you already used this approach above, use URL-hacking (arguably the easier of the two methods in this case).

In [49]:
def get_next_page(soup):
    
    # is there a next button
    next_button = soup.find('li', class_='next')
    if next_button:
        
        url = next_button.find('a').attrs['href']
    
        # if it's page 2 then we're ok
        # if it's a later page then we need to add catalogue/ to the url
    
        if not 'catalogue' in url:
            url = 'catalogue/' + url
    
        return f'http://books.toscrape.com/{url}'
    
    else: # last page
        return None
    
html_page = requests.get('http://books.toscrape.com/') # Make a get request to retrieve the page
soup = BeautifulSoup(html_page.content, 'html.parser') # Pass the page contents to beautiful soup for parsing

df = get_page_info(soup)

# get the next url
url = get_next_page(soup)

while url:
    html_page = requests.get(url) # Make a get request to retrieve the page
    
    soup = BeautifulSoup(html_page.content, 'html.parser') # Pass the page contents to beautiful soup for parsing
    
    df_new_page = get_page_info(soup)
    
    df = pd.concat([df, df_new_page])
    
    # get the next url
    url = get_next_page(soup)
# add the column names
df.columns = ['Title', 'Rating', 'Price', 'Avail']
    
df

,Title,Rating,Price,Avail
0,A Light in the Attic,3,51.77,In stock
1,Tipping the Velvet,1,53.74,In stock
2,Soumission,1,50.1,In stock
3,Sharp Objects,4,47.82,In stock
4,Sapiens: A Brief History of Humankind,5,54.23,In stock
...,...,...,...,...
15,Alice in Wonderland (Alice's Adventures in Won...,1,55.53,In stock
16,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",4,57.06,In stock
17,A Spy's Devotion (The Regency Spies of London #1),5,16.97,In stock
18,1st to Die (Women's Murder Club #1),1,53.98,In stock


## Summary

Well done! You just completed your first full web scraping project! You're ready to start harnessing the power of the web!